In [1]:
import pickle

df_sentiment_score=pickle.load(open('../src/data/sentiment_score_df.pkl','rb'))

In [2]:
df_sentiment_score.head()

,text,stars,embeddings,sentiment
0,Ate at the bar late one evening. The food was...,3.0,"[0.02110215, 0.015082884, 0.027772486, 0.04898...",0
1,ive gone here more than 10 times. the first f...,2.0,"[0.053978473, -0.005075182, 0.03789233, 0.0295...",-1
2,"Went here Saturday for drag brunch, we had 8 p...",3.0,"[0.05708591, -0.007079023, 0.02999837, 0.03157...",0
3,Turns out they randomly close the pool down wh...,3.0,"[0.01065718, 0.014424983, 0.042207252, 0.02751...",0
4,"Totally awesome experience. Very New Orleans ""...",3.0,"[0.029420882, 0.032237433, 0.026742732, 0.0395...",0


In [3]:
from pyathena import connect
import pandas as pd
import configparser

In [8]:
config = configparser.ConfigParser()
config.read('../access_keys_shreya.cfg')
AWS_ACCESS_KEY = config.get('aws', 'aws_access_key')
AWS_SECRET_KEY = config.get('aws', 'aws_secret_key')

In [9]:
from six.moves.urllib.parse import quote_plus
from sqlalchemy.engine import create_engine

SCHEMA_NAME = "yelp"
S3_STAGING_DIR = "s3://sairin.yelp.dataset/dataset"
AWS_REGION = "us-west-1"
conn_str = (
    "awsathena+rest://{aws_access_key_id}:{aws_secret_access_key}@"
    "athena.{region_name}.amazonaws.com:443/"
    "{schema_name}?s3_staging_dir={s3_staging_dir}&work_group=primary"
)


# Create the SQLAlchemy connection. Note that you need to have pyathena installed for this.
engine = create_engine(
    conn_str.format(
        aws_access_key_id=quote_plus(AWS_ACCESS_KEY),
        aws_secret_access_key=quote_plus(AWS_SECRET_KEY),
        region_name=AWS_REGION,
        schema_name=SCHEMA_NAME,
        s3_staging_dir=quote_plus(S3_STAGING_DIR),
    )
)
athena_connection = engine.connect()


In [10]:
query_business = """
        SELECT * 
        FROM yelp.philadelphia_top_restaurant
        """


In [15]:
runBlock = False
if runBlock:
    df_restaurant_philadelphia = pd.read_sql(query_business, athena_connection)
    df_restaurant_philadelphia.to_pickle('../src/data/top_restaurant_philadelphia.pkl')

In [16]:
df_restaurant_philadelphia = pd.read_pickle('../src/data/top_restaurant_philadelphia.pkl')

In [17]:
df_restaurant_philadelphia.head()

,review_id,business_id,stars,text,name,address,city,state,latitude,longitude
0,1znepeEBKLNu7xvzc4A0ww,eMiN8nm70jjKg8izikVWDA,4.0,What can I say!!!! Best Mussels anywhere!! T...,Chickie's & Pete's,1526 Packer Ave,Philadelphia,PA,39.911417,-75.174511
1,4SC_wghn0RR1ccXMEjYkuw,eMiN8nm70jjKg8izikVWDA,3.0,Love the fact that they made me turn my hat fo...,Chickie's & Pete's,1526 Packer Ave,Philadelphia,PA,39.911417,-75.174511
2,5ouqnL6XP-m6UYw_vmSx1Q,eMiN8nm70jjKg8izikVWDA,4.0,"Natives recommended I try the crab fries, whic...",Chickie's & Pete's,1526 Packer Ave,Philadelphia,PA,39.911417,-75.174511
3,kNDutSQQtNpxZcGpIkrGrA,eMiN8nm70jjKg8izikVWDA,5.0,This place is the real deal !! It's a true spo...,Chickie's & Pete's,1526 Packer Ave,Philadelphia,PA,39.911417,-75.174511
4,v9gWqobdJozPb8CFsqOPzQ,eMiN8nm70jjKg8izikVWDA,5.0,I didn't know what a crab fry was until I went...,Chickie's & Pete's,1526 Packer Ave,Philadelphia,PA,39.911417,-75.174511


In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sairindhri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
from sentence_transformers import SentenceTransformer
# import tqdm.notebook as tq
from tqdm import tqdm
from time import sleep
import numpy as np


# Load the pre-trained model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_mean_embedding(row):
    review = row
    review_sentances = nltk.sent_tokenize(review)

    embeddings = bert_model.encode(review_sentances)

    mean_embeddings =  np.mean(embeddings, axis=0)
    return mean_embeddings

def generate_bert_embeddings(review_text):
  
    mean_embeddings=generate_mean_embedding(review_text)
    return mean_embeddings
    



# df_balanced_data.to_pickle("intermediate_data/bert_generated_embeddings_balanced_dataframe.pkl")

In [49]:
text=df_restaurant_philadelphia['text'][0]
text

"What can I say!!!!   Best Mussels anywhere!! The only decision is whether to get Red or White... Last several visits .. Been RED!!!!!  I cant even bring myself to order Mussels anywhere else!!!! Latest visit yesterday My sons birthday coming from a Flyers game.. and excited to watch the Seahawks I'm a fan from way back and wanted to experience the win on Chickies huge TV!!  Of all my numerous visits I never realized they have a secret section all the way in the far corner with really nice cozy booths!!! Our waitress Melanie was very accommodating and also had the TVs in our section fixed for us to watch  the Football game. .. Her service was excellent .. The food was awesome including Clams, Mussels, roast pork and a parm sandwich everyone was pleased .. No Crab Fries this  time !! Maybe since I had to wait 30 minutes at the Eagles game in line freezing for them last Saturday!!!!!  Only complaint for the appetizer was not enough chips for the overwhelming toppings on the cheese steak 

In [30]:

text=df_restaurant_philadelphia['text'][0]
embedding=generate_bert_embeddings(text)
embedding

array([-2.91974582e-02, -6.99813943e-04,  2.03757659e-02, -2.43058372e-02,
        8.97509512e-03, -3.98457807e-04,  2.52164789e-02, -2.97515597e-02,
       -8.02864041e-03, -2.21344363e-02, -2.71888338e-02, -4.28020349e-03,
        1.86908413e-02, -5.50995534e-03,  1.31068274e-03, -3.63660529e-02,
        4.68011089e-02, -4.09303121e-02, -1.05011007e-02,  1.16370404e-02,
       -3.34994309e-02, -8.30564764e-04,  7.29405880e-03,  2.60480493e-02,
       -2.04771496e-02,  2.32169572e-02, -1.03646712e-02,  3.93238254e-02,
       -1.53775951e-02, -2.98258215e-02, -3.16025391e-02,  1.52241029e-02,
        7.46485218e-03,  5.40413242e-03,  7.75447465e-04, -1.31628038e-02,
        1.04650911e-02, -7.95988068e-02,  1.26506928e-02,  5.25024831e-02,
       -2.45453470e-04, -4.02689166e-03,  2.03990750e-02,  2.68069766e-02,
       -3.38193635e-03, -2.70393062e-02, -5.41956630e-03, -2.32448708e-02,
        3.97737324e-02,  1.51316123e-02,  9.97357350e-03,  4.35008481e-03,
        1.05099548e-02, -

In [50]:
def predict_sentiment_score(embedding):
    embedding_new = np.array(embedding).reshape(1, -1)
    model = pickle.load(open('../src/data/svm_sentiment_prediction_model.pkl','rb'))
    sentiment_score = model.predict(embedding_new)
    return sentiment_score


In [51]:
sentiment_score=predict_sentiment_score(embedding)
sentiment_score[0]


1

In [36]:
embedding.shape

(384,)